In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

In [2]:
import json
import time
import pickle
import numpy as np
from string import punctuation

In [3]:
from concurrent.futures import ThreadPoolExecutor

## Test Selenium

In [8]:
url = "https://www.glassdoor.com/Career/index.htm"
options = Options()
options.headless = False
options.add_argument('window-size=1400,600')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--allow-insecure-localhost")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")
options.add_argument("--remote-debugging-port=9222")
driver = webdriver.Chrome(executable_path='chromedriver.exe',options=options)
driver.get(url)
saved_cookies = pickle.load(open("cookies.pkl", "rb"))
driver.delete_all_cookies()
for cookie in saved_cookies:
    driver.add_cookie(cookie)


c:\anaconda3\envs\newenv\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # This is added back by InteractiveShellApp.init_path()


In [7]:
driver.quit()

In [12]:
search_box = driver.find_element(By.ID, 'career-home-keyword-input')
search_box.send_keys("Investor Accounting Analyst")

In [9]:
search_box.clear()

In [13]:
clear_button = driver.find_element(By.XPATH, '//button[@data-test="career-home-search-submit"]')
clear_button.click()

In [17]:
search_button = driver.find_element(By.ID, 'title-location-search-btn')
search_button.click()

In [24]:
driver.find_element(By.XPATH, '//button[text()="Read More"]').click()
driver.find_element(By.XPATH, '//div[@data-test="career-summary"]/div/div/div/p').text.replace("\n",". ")

"Teachers develop curriculum and lesson plans, and present those lessons to their students, individually or in groups. They track the progress of their students and create reports to inform parents about their progress. Teachers are responsible for maintaining a safe and respectful classroom environment in order to optimize student capabilities and develop their love of learning.. . Teachers are required to have a Bachelor's degree and a teaching credential. In order to obtain a teaching credential, a minimum number of hours in education coursework must be completed and an exam must be passed. Requirements may vary by state and type of school. Many teachers choose to advance their career by entering into administrative positions, like prinicipal or superintendent."

In [29]:
temp = [e.text for e in driver.find_elements(By.XPATH, '//div[@data-test="career-summary"]/div/div/div/div/div')]

['What responsibilities are common for Teacher jobs?',
 'Develop curriculum to meet developmental goals and instructional activities\nPrepare lesson materials, grade papers, manage the classroom, and evaluate and give feedback to students\nExercise open and respectful communication with families, co-workers, supervisors, and other school staff\nMeasure progress and accomplishments of students towards their academic achievements on a regular basis and provide progress reports as required\nCreate and keep records on attendance, accidents, incidents, and other noteworthy events in accordance with licensing guidelines and program policies\nAttend conferences and workshops in order to further advance professional development\nManage student behavior in accordance with outlined guidelines\nProvide a safe, thriving, and respectful environment for all students in the classroom and on the school campus',
 'What are the typical qualifications for Teacher jobs?',
 "Bachelor's degree from an accre

In [21]:
driver.find_element(By.XPATH, '//div[@data-test="career-salaries"]/div/div/div/div').text

'$45,746 /yr'

In [15]:
driver.find_element(By.XPATH, '//div[@data-testid="avg-salary-value"]').text

'$91,207'

In [20]:
element = driver.find_element(By.XPATH, '//a[@class="working-as__learn-more"]')
element.click()

In [35]:
[e.text for e in driver.find_elements(By.XPATH, '//div[h2[text()="Related Careers"]]/a/b')]

['Kindergarten Teacher',
 'Substitute Teacher',
 'Elementary School Teacher',
 'Preschool Teacher']

In [40]:
driver.find_element(By.XPATH, '//div[p/strong[starts-with(text(),"What skills do you need")]]/ul').text.split("\n")

['CPR First Aid',
 'CDA',
 'English',
 'Teaching Experience',
 'First Aid',
 'English Language',
 'CPR',
 'Interpersonal and Communication']

## Loop thru Titles

In [4]:
def wait_for_element(driver, locator, t=3):
    try:
        element = WebDriverWait(driver, t).until(
            EC.presence_of_element_located(locator)
        )
        return element
    except: return False

In [5]:
def shorten(txt):
    txt = txt.strip()
    if (txt[0] in punctuation):
        txt = txt.strip(txt[0])
        return txt
    if (txt[-1] in punctuation):
        txt = txt.strip(txt[-1])
        return txt
    return " ".join(txt.split(" ")[1:])

In [6]:
def driver_setup(headless=True):
    options = Options()
    options.headless = headless
    options.add_argument('window-size=1400,600')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--allow-insecure-localhost")
    #options.add_argument("--allow-running-insecure-content")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")
    #options.add_argument("--remote-debugging-port=9222")
    driver = webdriver.Chrome(options=options)
    return driver

In [7]:
with open("data/lightcast_all_titles.json") as file:
    title_list = json.loads(file.read())

In [8]:
len(title_list)

72893

In [9]:
taboo = ["Assistant","Intern","Support","Team","Lead"]

In [24]:
def Glassdoor_Crawl(title_list, driver):
    url = "https://www.glassdoor.com/Career/index.htm"
    for title_info in title_list:
        title_name = title_info["name"]
        title_name = title_name.replace(".Net","Dotnet")
        skip = False
        for word in taboo:
            if word in title_name:
                skip = True
                break
        if skip:
            continue
            
        print(f"[Log]Crawling {title_name}")

        #print(f"{len(checkpoint)/len(title_list)*100:.2f}")
        driver.get(url)
        saved_cookies = pickle.load(open("cookies.pkl", "rb"))
        driver.delete_all_cookies()
        for cookie in saved_cookies:
            driver.add_cookie(cookie)
        problem = 0
        # if server error wait for an hour
        if problem>=3:
            print(f"[Log]Skipped {problem} times... Sleeping")
            #time.sleep(720)
            print(f"[Log]Restarting...")
            continue

        #search box
        search_box = driver.find_element(By.ID, 'career-home-keyword-input')
        #clear search
        search_box.clear()
        # clear + search for title name
        search_key = title_name
        search_box.send_keys(search_key)

        #search button
        search_button = driver.find_element(By.XPATH, '//button[@data-test="career-home-search-submit"]')
        search_button.click()

        #check if page is loaded
        if not wait_for_element(driver, (By.XPATH, '//a[@data-test="site-header-overview"]'), t=5):
            print("[Log]Cannot Load Overview Header")
            continue
        
        while not wait_for_element(driver, (By.XPATH, '//div[@data-test="career-summary"]/div'),t=2):
            if len(search_key.split(" "))<=1:
                    print(f"[Log]All search keys are invalid...")
                    skip = True
                    break
            search_key = shorten(search_key)
            print(f"[Log]Search key invalid...shortening to {search_key}")
            driver.find_element(By.NAME, 'career-discovery-zero-data').send_keys(search_key)
            driver.find_element(By.XPATH, '//button[span[text()="Search"]]').click()
            continue
        if skip:
            continue
        try:
            driver.find_element(By.XPATH, '//button[text()="Read More"]').click()
        except: pass

        if not wait_for_element(driver, (By.XPATH, '//div[@data-test="career-summary"]/div/div/div/p')):
            descrpt = None
        else:
            descrpt = driver.find_element(By.XPATH, '//div[@data-test="career-summary"]/div/div/div/p').text.replace("\n",". ")
            #temp = [e.text for e in driver.find_elements(By.XPATH, '//div[@data-test="career-summary"]/div/div/div/div/div')]
            
        Title = search_key

        if not wait_for_element(driver,(By.XPATH, '//div[@data-test="career-salaries"]/div/div/div/div')):
            salary = None
        else:
            salary = driver.find_element(By.XPATH, '//div[@data-test="career-salaries"]/div/div/div/div').text
        related_titles = [e.text for e in driver.find_elements(By.XPATH, '//div[h2[text()="Related Careers"]]/a/b')]

        #change page
        career_advice_tab = driver.find_element(By.XPATH, '//a[@data-test="site-header-careerPath"]')
        career_advice_tab.click()

        if not wait_for_element(driver, (By.XPATH, '//div[p/strong[starts-with(text(),"What skills do you need")]]/ul')):
            print("[Log]Cannot Load Required Skill")
            required_skills = []
        else:
            required_skills = driver.find_element(By.XPATH, '//div[p/strong[starts-with(text(),"What skills do you need")]]/ul').text.split("\n")

        title_info["Indeed_record"] = {}
        title_info["Indeed_record"]["name"] = Title
        title_info["Indeed_record"]["description"] = descrpt
        title_info["Indeed_record"]["salary"] = salary
        title_info["Indeed_record"]["related_titles"] = related_titles
        title_info["Indeed_record"]["required_skills"] = required_skills
        
        # save
        with open("glassdoor_title_extract.jsonl","a") as file:
            json.dump(title_info,file)
            file.write("\n")
        print(f"[Log]{title_name} Success!")
        # checkpoint
        with open("glassdoor_title_checkpoint.txt","a") as file:
            file.write(title_name+"\n")
        time.sleep(3)

In [32]:
with open("glassdoor_title_checkpoint.txt") as file:
  checkpoint = set(file.read().strip().split("\n"))

In [33]:
title_list = [i for i in title_list if i["name"] not in checkpoint]

In [34]:
N = 5
drivers = [driver_setup() for _ in range(N)]
chunks = np.array_split(title_list,N)

In [35]:
with ThreadPoolExecutor(max_workers=N) as executor:
    executor.map(Glassdoor_Crawl, chunks, drivers)
    print("Done")
[driver.quit() for driver in drivers]

[Log]Crawling AML Officer
[Log]Crawling Crisis Advocate
[Log]Crawling Food and Beverage Attendant/Runner
[Log]Crawling Messaging Specialist
[Log]Crawling SCIS Supervisor
Done
[Log]Search key invalid...shortening to and Beverage Attendant/Runner
[Log]Cannot Load Overview Header
[Log]Crawling SE Manager
[Log]Cannot Load Overview Header
[Log]Crawling Met Supervisor
[Log]Cannot Load Overview Header
[Log]Crawling AML/BSA Consultant
[Log]Search key invalid...shortening to Beverage Attendant/Runner
[Log]Cannot Load Required Skill
[Log]Crisis Advocate Success!
[Log]Search key invalid...shortening to Attendant/Runner
[Log]Crawling Crisis Intervention Advocate
[Log]All search keys are invalid...
[Log]Crawling Foot and Ankle Specialist
[Log]Cannot Load Overview Header
[Log]Crawling AP Reader
[Log]Cannot Load Overview Header
[Log]Crawling SEC Manager
[Log]Cannot Load Overview Header
[Log]Crawling Metabolic Specialist
[Log]Cannot Load Overview Header
[Log]Crawling Football Manager
[Log]Cannot Load 

[None, None, None, None, None]

In [43]:
[driver.quit() for driver in drivers]

[None, None, None]

In [ ]:
Glassdoor_Crawl(title_list, driver_setup(headless=False))